In [21]:
import numpy as np
import matplotlib.pyplot as plt
from assignment_3_helper import LCPSolve, assignment_3_render


# DEFINE GLOBAL PARAMETERS
L = 0.4
MU = 0.3
EP = 0.5
dt = 0.01
m = 0.3
g = np.array([0., -9.81, 0.])
rg = 1./12. * (2 * L * L) #TODO: Rename this to rg_squared since it is $$r_g^2$$ - Do it also in the master
M = np.array([[m, 0, 0], [0, m, 0], [0, 0, m * rg]])
Mi = np.array([[1./m, 0, 0], [0, 1./m, 0], [0, 0, 1./(m * rg)]])
DELTA = 0.001
T = 150


In [22]:
def get_contacts(q):
    """
        Return jacobian of the lowest corner of the square and distance to contact
        :param q: <np.array> current configuration of the object
        :return: <np.array>, <float> jacobian and distance
    """
    # ------------------------------------------------
    # FILL WITH YOUR CODE

    jac = None  # TODO: Replace None with your result
    phi = None

    x,y,theta = q[0],q[1],q[2]

    n_0 = np.array([[-1,-1, 0],
                  [1,-1, 0],
                  [1,1, 0],
                  [-1,1, 0]])
    r_0 = np.array([[-1*L/2,-1*L/2, 1],
                  [1*L/2,-1*L/2, 1],
                  [1*L/2,1*L/2, 1],
                  [-1*L/2,1*L/2, 1]])

    # print(r_0.T)
    w_R_0 = np.array([[np.cos(theta), -np.sin(theta), x],
                  [np.sin(theta), np.cos(theta), y],
                  [0,0,0]])
    # print(w_R_0)
    
    r_w = np.dot(w_R_0,r_0.T)[:-1, :]
    # print(r_w)
    n_w = np.dot(w_R_0,n_0.T)[:-1, :]
    # print(n_w)
    
    ind = np.argmin(r_w[-1])
    
    r_x,r_y = r_w[:,ind][0], r_w[:,ind][1]
    n_x,n_y = n_w[:,ind][0], n_w[:,ind][1]

    jac = np.array([[n_y, n_x],
                  [-n_x, n_y],
                  [-r_x*n_x - r_y*n_y , r_x*n_y - r_y*n_x]])
    phi = r_w[:,ind][1]
    # ------------------------------------------------
    return jac, phi

jac,phi = get_contacts(np.array([2,2,np.pi/4]))
print("Contact Jacobian:")
print(jac)
print("Distance to contact:")
print(phi)

Contact Jacobian:
[[-1.41421356  0.        ]
 [-0.         -1.41421356]
 [ 2.42842712 -2.82842712]]
Distance to contact:
1.7171572875253809


In [23]:
import numpy as np

def get_contacts(q):
    """
    Return jacobian of the lowest corner of the square and distance to contact
    :param q: <np.array> current configuration of the object (qt = [x, y, theta])
    :return: <np.array>, <float> jacobian and distance
    """
    
    # Dimensions of the square (side length)
    a = 1.0
    
    # Corner positions relative to the center
    corners_relative = np.array([[-a/2, -a/2],
                                 [a/2, -a/2],
                                 [-a/2, a/2],
                                 [a/2, a/2]])

    # Calculate rotation matrix
    theta = q[2]
    R = np.array([[np.cos(theta), -np.sin(theta)],
                  [np.sin(theta), np.cos(theta)]])
    
    # Calculate corner positions in world frame
    corners_world = q[:2] + np.dot(corners_relative, R.T)

    # Find the corner with the smallest y-coordinate (closest to the surface)
    min_corner = np.argmin(corners_world[:, 1])
    
    # Calculate the distance to the surface
    phi = q[1] - corners_world[min_corner, 1]

    # Calculate the contact Jacobian
    Jt = np.array([-np.sin(theta), np.cos(theta)])  # Tangential component
    Jn = np.array([0.0, -1.0])  # Normal component

    jac = np.column_stack((Jt, Jn))

    return jac, phi

# Example usage
jac, phi = get_contacts(np.array([2, 2, np.pi/4]))
print("Contact Jacobian:")
print(jac)
print("Distance to contact:")
print(phi)


Contact Jacobian:
[[-0.70710678  0.        ]
 [ 0.70710678 -1.        ]]
Distance to contact:
0.7071067811865475


In [24]:
def get_contacts(q):
    """
        Return jacobian of the lowest corner of the square and distance to contact
        :param q: <np.array> current configuration of the object
        :return: <np.array>, <float> jacobian and distance
    """
    # ------------------------------------------------
    # FILL WITH YOUR CODE

    jac = None  # TODO: Replace None with your result
    phi = None

    x,y,theta = q[0],q[1],q[2]
    
    r_0 = np.array([[-1*L/2,-1*L/2, 1],
                  [1*L/2,-1*L/2, 1],
                  [1*L/2,1*L/2, 1],
                  [-1*L/2,1*L/2, 1]])

    w_R_0 = np.array([[np.cos(theta), -np.sin(theta), x],
                  [np.sin(theta), np.cos(theta), y],
                  [0,0,0]])
    
    r_w = (np.dot(w_R_0,r_0.T)[:-1, :]).T

    #print(r_w)

    min_idx = np.argmin(r_w[:, 1])
    closest_corner = r_w[min_idx]
    
    phi = closest_corner[1]
    dx = closest_corner[0] - x
    dy = closest_corner[1] - y
    J_t = np.array([1, 0, -dy]) 
    J_n = np.array([0, 1, dx])
    jac = np.vstack((J_t, J_n)).T 
    # ------------------------------------------------
    return jac, phi

jac,phi = get_contacts(np.array([0,0,np.pi/4]))
print("Contact Jacobian:")
print(jac)
print("Distance to contact:")
print(phi)

Contact Jacobian:
[[ 1.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00]
 [ 2.82842712e-01 -8.80288308e-18]]
Distance to contact:
-0.28284271247461906


In [25]:
def get_contacts(q):
    """
        Return jacobian of the lowest corner of the square and distance to contact
        :param q: <np.array> current configuration of the object
        :return: <np.array>, <float> jacobian and distance
    """    
    x, y, theta = q
    l = 0.4  
    
    corners = np.array([
        [x + l/2 * np.cos(theta) - l/2 * np.sin(theta), y + l/2 * np.sin(theta) + l/2 * np.cos(theta)],
        [x - l/2 * np.cos(theta) - l/2 * np.sin(theta), y - l/2 * np.sin(theta) + l/2 * np.cos(theta)],
        [x + l/2 * np.cos(theta) + l/2 * np.sin(theta), y + l/2 * np.sin(theta) - l/2 * np.cos(theta)],
        [x - l/2 * np.cos(theta) + l/2 * np.sin(theta), y - l/2 * np.sin(theta) - l/2 * np.cos(theta)]
    ])
    print(corners)
    min_idx = np.argmin(corners[:, 1])
    closest_corner = corners[min_idx]
    
    phi = closest_corner[1]
    dx = closest_corner[0] - x
    dy = closest_corner[1] - y
    J_t = np.array([1, 0, -dy]) 
    J_n = np.array([0, 1, dx])
    jac = np.vstack((J_t, J_n)).T 
    
    return jac, phi

jac,phi = get_contacts(np.array([1,1,np.pi/4]))
print("Contact Jacobian:")
print(jac)
print("Distance to contact:")
print(phi)

[[1.         1.28284271]
 [0.71715729 1.        ]
 [1.28284271 1.        ]
 [1.         0.71715729]]
Contact Jacobian:
[[1.         0.        ]
 [0.         1.        ]
 [0.28284271 0.        ]]
Distance to contact:
0.7171572875253809


In [26]:
# DEFINE GLOBAL PARAMETERS
L = 0.4
MU = 0.3
EP = 0.5
dt = 0.01
m = 0.3
g = np.array([0., -9.81, 0.])
rg = 1./12. * (2 * L * L) #TODO: Rename this to rg_squared since it is $$r_g^2$$ - Do it also in the master
M = np.array([[m, 0, 0], [0, m, 0], [0, 0, m * rg]])
Mi = np.array([[1./m, 0, 0], [0, 1./m, 0], [0, 0, 1./(m * rg)]])
DELTA = 0.001
T = 150

def form_lcp(jac, v):
    """
        Return LCP matrix and vector for the contact
        :param jac: <np.array> jacobian of the contact point
        :param v: <np.array> velocity of the center of mass
        :return: <np.array>, <np.array> V and p
    """
    # ------------------------------------------------
    # FILL WITH YOUR CODE

    V = None  # TODO: Replace None with your result
    p = None
    
    J_hat = np.block([jac[:,1:2], -jac[:,0:1], jac[:,0:1]]) # 3 x 3
    #print(np.shape(J_hat))
    V = np.matmul(J_hat.T,M*dt) # 3 x 3
    
    V = np.block([[V, np.array([[0],[1],[1]])],
                  [MU, -1, -1, 0]])
    # f_e = np.array([[0],
    #                 [-m*9.81],
    #                 [0]])
    f_e = np.matmul(M,g).reshape((3,1))
    
    J_n, _, J_t = jac[:,1:2], -jac[:,0:1], jac[:,0:1]
    #print(np.shape(J_n))
    v_t = v.reshape((3,1)) # (3,)

    # (3x1) * ((3x1) + 
    #print(np.shape((1+EP)*v_t + dt*np.matmul(Mi,f_e)))
    row_1 = np.matmul(J_n.T,(1+EP)*v_t + dt*np.matmul(Mi,f_e)).reshape(-1)
    row_2 = np.matmul(-J_t.T,v_t + dt*np.matmul(Mi,f_e)).reshape(-1)
    row_3 = -row_2

    p = np.vstack((row_1, row_2, row_3, 0)).reshape(-1)

    # ------------------------------------------------
    return V, p

v =  np.array([[1],
               [1],
               [1]])
               
V,p = form_lcp(jac,v.reshape(-1))
print("Shape of V: ",V)
print("Shape of p: ", p)


Shape of V:  [[ 0.0000000e+00  3.0000000e-03  0.0000000e+00  0.0000000e+00]
 [-3.0000000e-03  0.0000000e+00 -2.2627417e-05  1.0000000e+00]
 [ 3.0000000e-03  0.0000000e+00  2.2627417e-05  1.0000000e+00]
 [ 3.0000000e-01 -1.0000000e+00 -1.0000000e+00  0.0000000e+00]]
Shape of p:  [ 1.4019     -1.28284271  1.28284271  0.        ]


In [27]:
# DEFINE GLOBAL PARAMETERS
L = 0.4
MU = 0.3
EP = 0.5
dt = 0.01
m = 0.3
g = np.array([0., -9.81, 0.])
rg = 1./12. * (2 * L * L) #TODO: Rename this to rg_squared since it is $$r_g^2$$ - Do it also in the master
M = np.array([[m, 0, 0], [0, m, 0], [0, 0, m * rg]])
Mi = np.array([[1./m, 0, 0], [0, 1./m, 0], [0, 0, 1./(m * rg)]])
DELTA = 0.001
T = 150

def form_lcp(jac, v):
    """
        Return LCP matrix and vector for the contact
        :param jac: <np.array> jacobian of the contact point
        :param v: <np.array> velocity of the center of mass
        :return: <np.array>, <np.array> V and p
    """
    # ------------------------------------------------
    # FILL WITH YOUR CODE

    Jt = jac[:, 0].reshape(-1, 1)  
    Jn = jac[:, 1].reshape(-1, 1)  

    M_inv = Mi

    JnT_M_inv = np.matmul(Jn.T, M_inv)
    JtT_M_inv = np.matmul(Jt.T, M_inv)

    JnT_M_inv_Jn = np.matmul(JnT_M_inv, Jn)
    JnT_M_inv_Jt = np.matmul(JnT_M_inv, Jt)
    JtT_M_inv_Jn = np.matmul(JtT_M_inv, Jn)
    JtT_M_inv_Jt = np.matmul(JtT_M_inv, Jt)

    V = np.array([
        [JnT_M_inv_Jn[0][0] * dt, -JnT_M_inv_Jt[0][0] * dt, JnT_M_inv_Jt[0][0] * dt, 0],
        [-JtT_M_inv_Jn[0][0] * dt, JtT_M_inv_Jt[0][0] * dt, -JtT_M_inv_Jt[0][0] * dt, 1],
        [JtT_M_inv_Jn[0][0] * dt, -JtT_M_inv_Jt[0][0] * dt, JtT_M_inv_Jt[0][0] * dt, 1],
        [MU, -1, -1, 0]
    ])

    fe = m * g  
    JnT_M_inv_fe = np.matmul(JnT_M_inv, fe)
    JtT_M_inv_fe = np.matmul(JtT_M_inv, fe)

    p1 = (np.matmul(Jn.T, v) * (1 + EP) + dt * JnT_M_inv_fe).item()
    p2 = (-np.matmul(Jt.T, v) + dt * JtT_M_inv_fe).item()
    p3 = (np.matmul(Jt.T, v) + dt * JtT_M_inv_fe).item()

    p = np.array([p1, p2, p3, 0])
    return V,p

v =  np.array([[1],
               [1],
               [1]])
               

V,p = form_lcp(jac,v.reshape(-1))
print("Shape of V: ",V.dtype)
print("Shape of p: ", p.dtype)

Shape of V:  float64
Shape of p:  float64


In [28]:
def step(q, v):
    """
        predict next config and velocity given the current values
        :param q: <np.array> current configuration of the object
        :param v: <np.array> current velocity of the object
        :return: <np.array>, <np.array> q_next and v_next
    """
    # ------------------------------------------------
    # FILL WITH YOUR CODE

    f_e = m*g  
    
    jac, phi = get_contacts(q)

    if phi <= DELTA:  # Within contact threshold
        V, p = form_lcp(jac, v)
        f_c = lcp_solve(V, p)

        # Update velocity and position
        v_next = v + dt * np.matmul(Mi, f_e+ jac[:,1]*f_c[0] - jac[:,0]*f_c[1] + jac[:,0]*f_c[2])
        q_next = q + dt * v_next + np.array([0, DELTA, 0]) 

    else:
        # If not in contact, just f_e
        a = np.dot(Mi, f_e)
        v_next = v + dt * a
        q_next = q + dt * v_next

    # ------------------------------------------------
    return q_next, v_next

def simulate(q0, v0):
    """
        predict next config and velocity given the current values
        :param q0: <np.array> initial configuration of the object
        :param v0: <np.array> initial velocity of the object
        :return: <np.array>, <np.array> q and v trajectory of the object
    """
    # ------------------------------------------------
    # FILL WITH YOUR CODE

    q = np.zeros((3, T))  # TODO: Replace with your result
    v = np.zeros((3, T))
    
    q[:, 0] = q0
    v[:, 0] = v0
    
    for i in range(1, T):
        q[:, i], v[:, i] = step(q[:, i-1], v[:, i-1])
    # ------------------------------------------------
    return q, v




In [29]:
def lcp_solve(V, p):
    """
        DO NOT CHANGE -- solves the LCP
        :param V: <np.array> matrix of the LCP
        :param p: <np.array> vector of the LCP
        :return: renders the trajectory
    """
    sol = LCPSolve(V, p)
    f_r = sol[1][:3]
    return f_r


def render(q):
    """
        DO NOT CHANGE -- renders the trajectory
        :param q: <np.array> configuration trajectory
        :return: renders the trajectory
    """
    assignment_3_render(q)


if __name__ == "__main__":
    # to test your final code, use the following initial configs
    q0 = np.array([0.0, 1.5, np.pi / 180. * 30.])
    v0 = np.array([0., -0.2, 0.])
    q, v = simulate(q0, v0)

    plt.plot(q[1, :])
    plt.show()

    render(q)

[[ 0.07320508  1.77320508]
 [-0.27320508  1.57320508]
 [ 0.27320508  1.42679492]
 [-0.07320508  1.22679492]]
[[ 0.07320508  1.77022408]
 [-0.27320508  1.57022408]
 [ 0.27320508  1.42381392]
 [-0.07320508  1.22381392]]
[[ 0.07320508  1.76626208]
 [-0.27320508  1.56626208]
 [ 0.27320508  1.41985192]
 [-0.07320508  1.21985192]]
[[ 0.07320508  1.76131908]
 [-0.27320508  1.56131908]
 [ 0.27320508  1.41490892]
 [-0.07320508  1.21490892]]
[[ 0.07320508  1.75539508]
 [-0.27320508  1.55539508]
 [ 0.27320508  1.40898492]
 [-0.07320508  1.20898492]]
[[ 0.07320508  1.74849008]
 [-0.27320508  1.54849008]
 [ 0.27320508  1.40207992]
 [-0.07320508  1.20207992]]
[[ 0.07320508  1.74060408]
 [-0.27320508  1.54060408]
 [ 0.27320508  1.39419392]
 [-0.07320508  1.19419392]]
[[ 0.07320508  1.73173708]
 [-0.27320508  1.53173708]
 [ 0.27320508  1.38532692]
 [-0.07320508  1.18532692]]
[[ 0.07320508  1.72188908]
 [-0.27320508  1.52188908]
 [ 0.27320508  1.37547892]
 [-0.07320508  1.17547892]]
[[ 0.07320508  1.71

KeyboardInterrupt: 